# 🔥 Forest Fire Simulation using AI/ML
This notebook predicts and simulates the spread of forest fires using U-Net and Cellular Automata models.
Region: Uttarakhand | Resolution: 30m | Models: U-Net, Cellular Automata


In [ ]:
# 📦 Install Required Libraries
!pip install rasterio geopandas tensorflow matplotlib scikit-learn

## 🧠 U-Net Model for Fire Probability Map

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

def build_unet(input_shape=(128, 128, 10)):
    inputs = layers.Input(shape=input_shape)
    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    p1 = layers.MaxPooling2D((2, 2))(c1)

    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
    p2 = layers.MaxPooling2D((2, 2))(c2)

    c5 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
    c5 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c5)

    u6 = layers.UpSampling2D((2, 2))(c5)
    u6 = layers.concatenate([u6, c2])
    c6 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(u6)
    c6 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c6)

    u7 = layers.UpSampling2D((2, 2))(c6)
    u7 = layers.concatenate([u7, c1])
    c7 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(u7)
    c7 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c7)

    outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(c7)
    model = models.Model(inputs=[inputs], outputs=[outputs])
    return model

model = build_unet()
model.summary()

## 🔥 Cellular Automata for Fire Spread Simulation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def simulate_fire_spread(initial_fire_map, slope_map, fuel_map, hours=3):
    spread_map = np.copy(initial_fire_map)
    for hour in range(hours):
        new_map = spread_map.copy()
        for i in range(1, spread_map.shape[0]-1):
            for j in range(1, spread_map.shape[1]-1):
                if spread_map[i, j] == 1:
                    for dx in [-1, 0, 1]:
                        for dy in [-1, 0, 1]:
                            ni, nj = i + dx, j + dy
                            if spread_map[ni, nj] == 0:
                                fire_chance = 0.2 + 0.1 * slope_map[ni, nj] + 0.1 * fuel_map[ni, nj]
                                if np.random.rand() < fire_chance:
                                    new_map[ni, nj] = 1
        spread_map = new_map.copy()
    return spread_map